In [ ]:
#!conda install gdcm -c conda-forge -y

In [ ]:
import os
import pandas as pd
import numpy as np
from PIL import Image
import matplotlib.pylab as plt

import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut
from tqdm.auto import tqdm

import tensorflow as tf
#import tensorflow_io as tfio
import IPython.display as display

In [ ]:
df = pd.read_csv('../input/siim-covid19-detection/train_image_level.csv')
print(df.shape)
df.head()

In [ ]:
image_path = '../input/siim-covid19-detection/train/00086460a852/9e8302230c91/65761e66de9f.dcm'
#dicom = pydicom.read_file(image_path)
dicom_file = pydicom.dcmread(image_path)
PatientSex = dicom_file.PatientSex
PatientSex

In [ ]:
dicom_file

In [ ]:
img = apply_voi_lut(dicom_file.pixel_array, dicom_file)

if dicom_file.PhotometricInterpretation == "MONOCHROME1":
    img = np.amax(img) - img
    
    img = img - np.min(img)
    img = img / np.max(img)
    img = (img * 255).astype(np.uint8)
    
# rescale pixel value between 0 to 255
#img = (((img - np.min(img))/np.max(img))*255.0).astype(np.uint8)

#print(img.dtype)  #float64

plt.figure(figsize = (6,6))
plt.imshow(img, cmap=plt.cm.gray)
#plt.savefig("./train/png/" + image_id + "".png")

In [ ]:
# Original from: https://www.kaggle.com/haiyunhu/siim-covid-19-convert-to-jpg-256px/edit
def read_xray(path,voi_lut = True, fix_monochrome = True):
    #image_bytes = pydicom.read_file(path)
    dicom = pydicom.read_file(path)
    
    # VOI LUT (if available by DICOM device) is used to transform raw DICOM data to 
    # "human-friendly" view
    if voi_lut:
        img = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        img = dicom.pixel_array
               
    # depending on this value, X-ray may look inverted - fix that:
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        img = np.amax(img) - img
        
    img = (((img - np.min(img))/np.max(img))*255.0).astype(np.uint8)
    
    return img
        
def resize(img, size, keep_ratio=False, resample=Image.LANCZOS):
    img = Image.fromarray(img)  # no shape
    if keep_ratio:
        img.thumbnail((size, size), resample)
    else:
        img = img.resize((size, size), resample)
    
    return img
    

In [ ]:
# Original from: https://www.kaggle.com/haiyunhu/siim-covid-19-convert-to-jpg-256px/edit
image_id = []
dim0 = []
dim1 = []
splits = []

for split in ['test', 'train']:
    save_dir = f'/kaggle/tmp/{split}/'
    os.makedirs(save_dir, exist_ok=True)
    
    for dirname, _, filenames in tqdm(os.walk(f'../input/siim-covid19-detection/{split}')):
        
        for file in filenames:
            #path = os.path.join(dirname, file)
            #print(path)
            xray = read_xray(os.path.join(dirname, file))
            im = resize(xray, size=512)  
            im.save(os.path.join(save_dir, file.replace('dcm', 'png')))

            image_id.append(file.replace('.dcm', ''))
            dim0.append(xray.shape[0])
            dim1.append(xray.shape[1])
            splits.append(split)

In [ ]:
df = pd.DataFrame.from_dict({'image_id': image_id, 'dim0': dim0, 'dim1': dim1, 'split': splits})
df.to_csv('meta.csv', index=False)

In [ ]:
df

In [ ]:
%%time
!tar -zcf train.tar.gz -C "/kaggle/tmp/train/" .
!tar -zcf test.tar.gz -C "/kaggle/tmp/test/" .